# 원시 이미지 데이터 로드 기능 테스트

레이블 등 정보가 없는 raw image data에 대한 데이터셋 로드 기능 구현 테스트<br>
앱 실행 시 분기 처리 예정

## Load Dataset

from_images_dir 메서드 사용<br>
<br>
절차
* 이미지 로드
* 데이터 타입 태그 부여 (RawImageData)
* CLIP 임베딩 생성 및 저장
* 데이터셋 생성 및 로드


In [40]:
import fiftyone as fo

## 데이터셋 초기화
for d in fo.list_datasets():
    if d != 'kolp_dataset':
        fo.delete_dataset(d)
fo.list_datasets()

['kolp_dataset']

이미지 데이터 경로 지정<br>
데이터셋 명칭, 형식 태그 부여

In [22]:
dataset = fo.Dataset.from_images_dir('../datasets/test_raw_imgs/')
dataset.name = 'test_raw_imgs'
dataset.tags.append('RawImageData')

dataset

 100% |█████████████████████| 4/4 [8.4ms elapsed, 0s remaining, 478.1 samples/s]     


Name:        test_raw_imgs
Media type:  image
Num samples: 4
Persistent:  False
Tags:        ['raw_image']
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField

In [37]:
dataset = fo.load_dataset('test_raw_imgs')

dataset

Name:        test_raw_imgs
Media type:  image
Num samples: 4
Persistent:  True
Tags:        ['raw_image']
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    clip_embeddings:  fiftyone.core.fields.ListField(fiftyone.core.fields.FloatField)

CLIP embedding 부여 확인<br>
> 실제 임베딩 인코드 기능은 별도 구현되어 있음

In [38]:
dataset.first()['clip_embeddings']

[-0.412820428609848,
 0.13943465054035187,
 -0.31811290979385376,
 0.06706452369689941,
 0.16231833398342133,
 -0.2364383041858673,
 -0.12425051629543304,
 -0.5306825637817383,
 -0.1880384385585785,
 0.01504066027700901,
 0.30554714798927307,
 -0.015592722222208977,
 0.7258403301239014,
 0.19378788769245148,
 0.45029065012931824,
 -0.1531013697385788,
 0.6529099941253662,
 0.01911531388759613,
 -0.45625630021095276,
 0.2901841402053833,
 0.44986894726753235,
 -0.23095914721488953,
 0.3069247305393219,
 -0.17676334083080292,
 0.6081085801124573,
 0.061425406485795975,
 0.2112702578306198,
 -0.25599923729896545,
 -0.23414316773414612,
 -0.5000873804092407,
 0.0014935601502656937,
 0.0030513829551637173,
 -0.44987377524375916,
 0.2012314796447754,
 -0.30158838629722595,
 -0.03512710705399513,
 0.05395463481545448,
 -0.45087242126464844,
 -0.2542111575603485,
 1.0188714265823364,
 -0.1545545905828476,
 -0.03935543820261955,
 -0.037260912358760834,
 0.014460432343184948,
 -0.168090760707855

## Labeler 연동

오픈소스 레이블링 툴 Label Studio 연동 테스트

In [1]:
import os

# 환경 변수 설정
os.environ['FIFTYONE_LABELSTUDIO_API_KEY'] = '3056e3590d27b004664c7eaa9222ac6410f50412'
os.environ['FIFTYONE_LABELSTUDIO_URL'] = 'http://localhost:8080'

# 설정된 환경 변수 확인
print(os.environ['FIFTYONE_LABELSTUDIO_API_KEY'])
print(os.environ['FIFTYONE_LABELSTUDIO_URL'])

3056e3590d27b004664c7eaa9222ac6410f50412
http://localhost:8080


In [11]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

dataset = fo.load_dataset('test_raw_imgs')
anno_key = 'test1'
classes = ['tank', 'truck', 'car']

print(os.environ.get('FIFTYONE_LABELSTUDIO_API_KEY'))
print(os.environ.get('FIFTYONE_LABELSTUDIO_URL'))

dataset.annotate(
    anno_key,
    backend='labelstudio',
    label_field='ground_truth',
    label_type='detections',
    launch_editor=True,
    classes=classes,
)

3056e3590d27b004664c7eaa9222ac6410f50412
http://localhost:8080
The backend 'labelstudio' does not support attributes. Provided attributes will be ignored.
Uploading media to Label Studio...
Upload complete
Launching editor at 'http://localhost:8080/projects/4'...


In [5]:
dataset.list_annotation_runs()

['test1']

In [10]:
dataset.delete_annotation_run(anno_key)